In [1]:
import os 
import pandas as pd
import shutil

# Explore data

In [2]:
df = pd.read_csv('./data/HAM10000_metadata.csv')

In [3]:
df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [4]:
cls = list(df['dx'].unique())
cls

['bkl', 'nv', 'df', 'mel', 'vasc', 'bcc', 'akiec']

In [5]:
for cl in cls:
    l = df[df['dx']==cl].shape[0]
    print(f'{cl}: {l}')

bkl: 1099
nv: 6705
df: 115
mel: 1113
vasc: 142
bcc: 514
akiec: 327


# Create directory of original files

The lines of code in this paragraph are not necessary for the model.

In [6]:
os.makedirs('./classes', exist_ok=True)

In [7]:
# We make directories for each type of lesion
for cl in cls:
    os.makedirs(f'./classes/{cl}', exist_ok=True)

In [10]:
# We copy each file to the folder of the right type of lesion
df_2 = df.set_index('image_id')
for img in os.listdir('./data/HAM10000_images_part_1'):
    cl = df_2.loc[img.rstrip('.jpg'), 'dx']
    shutil.copy(f'./data/HAM10000_images_part_1/{img}', f'./classes/{cl}/{img}')
for img in os.listdir('./data/HAM10000_images_part_2'):
    cl = df_2.loc[img.rstrip('.jpg'), 'dx']
    shutil.copy(f'./data/HAM10000_images_part_2/{img}', f'./classes/{cl}/{img}')

# balancing the classes

In [8]:
from sklearn.utils import resample
from sklearn.model_selection import train_test_split

In [19]:
# We split the data int a train, validation and test set according to the following odds (8:1:1)
X_train, X_test = train_test_split(df, train_size=0.8, stratify=df['dx'], random_state=76)

X_val, X_test = train_test_split(X_test, train_size=0.5, random_state=76, stratify=X_test['dx'])

In [20]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

(8012, 7)
(1001, 7)
(1002, 7)


In [25]:
# In the train and validation set we upsample the classes.
# After this operation each class has the same number of images
S_train = pd.DataFrame(columns=df.columns)
S_val = pd.DataFrame(columns=df.columns)
for cl in cls:
    n_samples_train = 5364
    n_samples_val = 670
    frame = X_train[X_train['dx']==cl]
    n_train = frame.shape[0]
    while n_samples_train > n_train:
        S_train = pd.concat([S_train, frame])
        n_samples_train -= n_train
    S_train = pd.concat([S_train, resample(frame, replace=False, n_samples=n_samples_train)])

    frame_val = X_val[X_val['dx']==cl]
    n_val = frame_val.shape[0]
    while n_samples_val > n_val:
        S_val = pd.concat([S_val, frame_val])
        n_samples_val -= n_val
    S_val = pd.concat([S_val, resample(frame_val, replace=False, n_samples=n_samples_val)])

## Putting classes into directories

In [13]:
The 
def create_directory(train_test: str, frame: pd.DataFrame()) -> list:
    '''
    Function that copies the images mentioned in the DataFrame into a folder inside the directory ./sampled_classes.
    These folders represent each type of lesion and if it comes form the train, test or validation set. 
    If everything went alright it will print as message. 
    If not it returns a list of images where the operation failed.
    
    :param train_test: str that must be either 'train','test' or 'validation'. Defines which subfolder the images must be put into.
    :param frame: pd.DataFrame with where the id of each image can be found.
    '''
    left = []
    cls = df['dx'].unique()
    for cl in cls:
        shutil.rmtree(f'./sampled_classes/{train_test}/{cl}', ignore_errors=True)
        os.makedirs(f'./sampled_classes/{train_test}/{cl}', exist_ok=True)
    i = 0
    for img, cl in zip(frame['image_id'], frame['dx']):
        if img+'.jpg' in os.listdir('./data/HAM10000_images_part_1'):
            shutil.copy(f'./data/HAM10000_images_part_1/{img}.jpg', f'./sampled_classes/{train_test}/{cl}/{img}_{str(i)}.jpg')
        elif img+'.jpg' in os.listdir('./data/HAM10000_images_part_2'):
            shutil.copy(f'./data/HAM10000_images_part_2/{img}.jpg', f'./sampled_classes/{train_test}/{cl}/{img}_{str(i)}.jpg')
        else:
            left.append(img)
        i += 1
    if len(left) == 0:
        print('Everything alright')
    else:
        return left

In [26]:
# we create the directory ./sampled_classes that the neural net will need.

shutil.rmtree('./sampled_classes')

os.makedirs('./sampled_classes/train', exist_ok=True)
os.makedirs('./sampled_classes/test', exist_ok=True)
os.makedirs('./sampled_classes/validation', exist_ok=True)

create_directory('train', S_train)
create_directory('validation', S_val)
create_directory('test', X_test)

Everything alright
Everything alright
Everything alright
